In [87]:
import folium
import geopandas as gpd
import pandas as pd
import requests
import json

# Baca file
df = pd.read_csv("sample_data/covid_19_indonesia_time_series_all.csv")
df.head()

,Date,Location ISO Code,Location,New Cases,New Deaths,New Recovered,New Active Cases,Total Cases,Total Deaths,Total Recovered,...,Latitude,New Cases per Million,Total Cases per Million,New Deaths per Million,Total Deaths per Million,Total Deaths per 100rb,Case Fatality Rate,Case Recovered Rate,Growth Factor of New Cases,Growth Factor of New Deaths
0,3/1/2020,ID-JK,DKI Jakarta,2,0,0,2,39,20,75,...,-6.204699,0.18,3.60,0.0,1.84,0.18,51.28%,192.31%,NaN,NaN
1,3/2/2020,ID-JK,DKI Jakarta,2,0,0,2,41,20,75,...,-6.204699,0.18,3.78,0.0,1.84,0.18,48.78%,182.93%,1.0,1.0
2,3/2/2020,IDN,Indonesia,2,0,0,2,2,0,0,...,-0.789275,0.01,0.01,0.0,0.00,0.00,0.00%,0.00%,NaN,NaN
3,3/2/2020,ID-RI,Riau,1,0,0,1,1,0,1,...,0.511648,0.16,0.16,0.0,0.00,0.00,0.00%,100.00%,NaN,NaN
4,3/3/2020,ID-JK,DKI Jakarta,2,0,0,2,43,20,75,...,-6.204699,0.18,3.96,0.0,1.84,0.18,46.51%,174.42%,1.0,1.0


In [88]:
# Konversi kolom tanggal ke format datetime
df['Date'] = pd.to_datetime(df['Date'])

# Filter data mulai dari 1 Mei 2021 sampai 16 September 2022
filtered_df = df[(df['Date'] >= '2021-05-01') & (df['Date'] <= '2022-09-09')]

In [89]:
# Pastikan kolom 'Date' dalam format datetime
df['Date'] = pd.to_datetime(df['Date'])

# Hapus data sebelum 1 Mei 2021
df = df[df['Date'] >= '2021-05-01']

# Cek hasil
df.head()

,Date,Location ISO Code,Location,New Cases,New Deaths,New Recovered,New Active Cases,Total Cases,Total Deaths,Total Recovered,...,Latitude,New Cases per Million,Total Cases per Million,New Deaths per Million,Total Deaths per Million,Total Deaths per 100rb,Case Fatality Rate,Case Recovered Rate,Growth Factor of New Cases,Growth Factor of New Deaths
14218,2021-05-01,ID-AC,Aceh,124,2,118,4,11169,441,9728,...,4.225615,23.63,2128.54,0.38,84.04,8.40,3.95%,87.10%,1.59,2.00
14219,2021-05-01,ID-BA,Bali,142,10,165,-33,44813,1346,42135,...,-8.369472,33.68,10628.84,2.37,319.25,31.92,3.00%,94.02%,1.04,1.67
14220,2021-05-01,ID-BT,Banten,53,0,28,25,47396,1210,44239,...,-6.456736,4.94,4420.29,0.00,112.85,11.28,2.55%,93.34%,0.74,0.00
14221,2021-05-01,ID-BE,Bengkulu,102,0,40,62,6884,172,5884,...,-3.533584,51.01,3442.79,0.00,86.02,8.60,2.50%,85.47%,2.37,1.00
14222,2021-05-01,ID-JK,DKI Jakarta,926,15,793,118,409540,6681,395761,...,-6.204699,85.38,37759.04,1.38,615.98,61.60,1.63%,96.64%,1.17,3.75


In [90]:
# Pastikan kolom Date dalam format datetime
df['Date'] = pd.to_datetime(df['Date'])

# Filter data dari 1 Mei 2021 sampai 16 September 2022
filtered_df = df[(df['Date'] >= '2021-05-01') & (df['Date'] <= '2022-09-16')]

# Hitung total kasus aktif (positif) per provinsi
total_kasus_per_provinsi = filtered_df.groupby('Province')['Total Active Cases'].sum().reset_index()

# Urutkan dari yang tertinggi
total_kasus_per_provinsi = total_kasus_per_provinsi.sort_values(by='Total Active Cases', ascending=False)

# Tampilkan hasil
print(total_kasus_per_provinsi)

                      Province  Total Active Cases
8                   Jawa Barat            14485629
4                  DKI Jakarta             8037103
9                  Jawa Tengah             6267172
2                       Banten             3627638
10                  Jawa Timur             3263680
5   Daerah Istimewa Yogyakarta             3242520
23                       Papua             2394030
33              Sumatera Utara             2112667
14            Kalimantan Timur             1585917
18                     Lampung             1328405
1                         Bali             1276032
25                        Riau             1229025
31              Sumatera Barat             1148957
27            Sulawesi Selatan             1135587
22         Nusa Tenggara Timur              965380
32            Sumatera Selatan              830362
17              Kepulauan Riau              791263
13           Kalimantan Tengah              791204
12          Kalimantan Selatan 

In [91]:
# Tampilkan 5 data teratas hasil filter
filtered_df.head()

# Load GeoJSON dari lokal
gdf = gpd.read_file("sample_data/indonesia-edit.geojson")
import json

with open("sample_data/indonesia-edit.geojson", "r", encoding="utf-8") as f:
    indonesia_geojson = json.load(f)

In [92]:
# Buat GeoDataFrame dari GeoJSON
geo = gpd.GeoDataFrame.from_features(indonesia_geojson["features"])

# Normalisasi nama kolom provinsi
geo['provinsi'] = geo['state'].str.strip().str.title()

In [93]:
# Data COVID baru sesuai data terbaru yang kamu berikan
data_baru = {
    'provinsi': [
        'Jawa Barat', 'Dki Jakarta', 'Jawa Tengah', 'Banten', 'Jawa Timur', 'Daerah Istimewa Yogyakarta', 'Papua',
        'Sumatera Utara', 'Kalimantan Timur', 'Lampung', 'Bali', 'Riau', 'Sumatera Barat', 'Sulawesi Selatan',
        'Nusa Tenggara Timur', 'Sumatera Selatan', 'Kepulauan Riau', 'Kalimantan Tengah', 'Kalimantan Selatan',
        'Aceh', 'Sulawesi Tengah', 'Kalimantan Utara', 'Sulawesi Utara', 'Kepulauan Bangka Belitung',
        'Kalimantan Barat', 'Jambi', 'Nusa Tenggara Barat', 'Bengkulu', 'Papua Barat', 'Sulawesi Tenggara',
        'Maluku', 'Maluku Utara', 'Sulawesi Barat', 'Gorontalo'
    ],
    'kasus': [
        14485629, 8037103, 6267172, 3627638, 3263680, 3242520, 2394030,
        2112667, 1585917, 1328405, 1276032, 1229025, 1148957, 1135587,
        965380, 830362, 791263, 791204, 764863,
        718933, 613144, 610052, 603046, 586654,
        509133, 457345, 398591, 374796, 271517, 262961,
        248984, 172020, 138308, 120769
    ]
}
df_covid = pd.DataFrame(data_baru)
df_covid['provinsi'] = df_covid['provinsi'].str.strip().str.title()

In [94]:
# Gabungkan data COVID dengan geo-data
geo_merged = geo.merge(df_covid, on='provinsi')

In [95]:
# uat peta dasar Indonesia
m = folium.Map(location=[-2.5, 118.0], zoom_start=5)

In [96]:
# Tambahkan marker tiap provinsi dengan info kasus
for _, row in geo_merged.iterrows():
    lat, lon = row['geometry'].centroid.y, row['geometry'].centroid.x
    popup = f"{row['provinsi']}: {row['kasus']:,} kasus"
    folium.Marker([lat, lon], popup=popup).add_to(m)

In [97]:
# Menampilkan peta
m